This notebook is to load predicted mask for hard-disk and calculate Segmentation evaluation metics.
predicted mask is the output of ScaleNASv2 Test and save predicted mask.ipynb

cp -v /mnt/fast-data/mjc/AutoRECIST/Codes/ScaleNAS/ScaleNASv1/tools/utils_test.py .

In [1]:
# coding: utf-8

# # This file is for segmetation metrics evaluation in 3D
# Edited by Jingchen around 06/20/2021
# This file is after ScaleNAS test which save predition into png images.
# This file load png images as predicted contours in 2D
# load cache pkl as gold-standard contours in 2D
# The stack 2D based on dicom-header to get 3D
# Evaluate 3D metics of dice, IoU, over-segmetation, and under-segmetation
# 
%reload_ext autoreload
%autoreload 2

import os
import logging
import numpy as np
import _init_paths

from config import cfg
from config import update_config
from utils_test import *

import cv2
from PIL import Image
import torch.nn.functional as F
from utils.utils import get_confusion_matrix
def convert_name(name):
    new = name.replace('/','_')
    return new

def get_palette(n):
    palette = [0] * (n * 3)
    for j in range(0, n):
        lab = j
        palette[j * 3 + 0] = 0
        palette[j * 3 + 1] = 0
        palette[j * 3 + 2] = 0
        i = 0
        while lab:
            palette[j * 3 + 0] |= (((lab >> 0) & 1) << (7 - i))
            palette[j * 3 + 1] |= (((lab >> 1) & 1) << (7 - i))
            palette[j * 3 + 2] |= (((lab >> 2) & 1) << (7 - i))
            i += 1
            lab >>= 3
    return palette
    
# def save_pred(preds, sv_path, name):

#     preds = preds.cpu().numpy().copy()
#     preds = np.asarray(np.argmax(preds, axis=1), dtype=np.uint8)
#     for i in range(preds.shape[0]):
#         cv2.imwrite(os.path.join(sv_path, convert_name(name[i])) , preds[i])

        
def save_pred( preds, sv_path, name):
    palette = get_palette(256)
    preds = preds.cpu().numpy().copy()
    preds = np.asarray(np.argmax(preds, axis=1), dtype=np.uint8)
    for i in range(preds.shape[0]):
        pred = preds[i]
        save_img = Image.fromarray(pred)
        save_img.putpalette(palette)
        save_img.save(os.path.join(sv_path, convert_name(name[i]) ))
        

def testval_lesion(config, test_dataset, testloader, model,
            sv_dir='', sv_pred=True, device = None):
    model.eval()
    confusion_matrix = np.zeros(
        (config.DATASET.NUM_CLASSES, config.DATASET.NUM_CLASSES))
    with torch.no_grad():
        for index, batch in enumerate(testloader):
            image, label, _, batchname = batch
            size = label.size()
            if device is None:
                image = image.cuda()
                label = label.long().cuda()
            else:
                image = image.to(device)
                label = label.long().to(device)

            pred = model(image)
            if pred.size()[-2] != size[-2] or pred.size()[-1] != size[-1]:
                pred = F.upsample(pred, (size[-2], size[-1]),
                                  mode='bilinear')

            confusion_matrix += get_confusion_matrix(
                label,
                pred,
                size,
                config.DATASET.NUM_CLASSES,
                config.TRAIN.IGNORE_LABEL)

            if sv_pred:
                sv_path = os.path.join(sv_dir, 'test_val_results')
                if not os.path.exists(sv_path):
                    os.mkdir(sv_path)
                save_pred(pred, sv_path, batchname)

            if index % 100 == 0:
                logging.info('processing: %d images' % index)
                pos = confusion_matrix.sum(1)
                res = confusion_matrix.sum(0)
                tp = np.diag(confusion_matrix)
                IoU_array = (tp / np.maximum(1.0, pos + res - tp))
                mean_IoU = IoU_array.mean()
                logging.info('mIoU: %.4f' % (mean_IoU))

    pos = confusion_matrix.sum(1)
    res = confusion_matrix.sum(0)
    tp = np.diag(confusion_matrix)
    pixel_acc = tp.sum() / pos.sum()
    mean_acc = (tp / np.maximum(1.0, pos)).mean()
    IoU_array = (tp / np.maximum(1.0, pos + res - tp))
    mean_IoU = IoU_array.mean()

    return mean_IoU, IoU_array, pixel_acc, mean_acc

In [2]:
cache_path = './cache/'
name = 'PDS_AMGEN_20020408_22Cat_test'
# name = 'lesion_train'

cache_filepath = os.path.join(cache_path, name+'_gt_roidb.pkl')
print('Loading cached gt_roidb from %s', cache_filepath)
with open(cache_filepath, 'rb') as fp:
    cached_roidb = pickle.load(fp)
    
roidb = cached_roidb


# In[25]:

from utils_test import __get_annotation__

sv_dir = mask_name = 'mask_1661'
sv_path = os.path.join(sv_dir, 'test_val_results')

all_boxes = [ [ np.zeros((0,5),dtype="float32") for _ in range(len(roidb)) ] for _ in range( cfg.DATASET.NUM_CLASSES) ]
all_segms = [ [ [] for _ in range(len(roidb)) ] for _ in range( cfg.DATASET.NUM_CLASSES) ]

for i in range(len(roidb)):
    if i%500==0:
        print('='*40,i)

    one = roidb[i]
    onename = one['image']
    pred_im = Image.open(os.path.join(sv_path, convert_name(onename) ))
    pred = np.array(pred_im)
    for j in range(cfg.DATASET.NUM_CLASSES):
        mask = np.asarray( pred==j , dtype=np.uint8)
        if np.sum(mask > 0) <= 3 :
            continue
        segmentation, bbox, area = __get_annotation__(mask , xywh = False , bbox_score=True)
        if segmentation and bbox:
            all_segms[j][i] = segmentation
            all_boxes[j][i] = bbox




site_ix = 8
gt_boxes = get_gt_boxes(cached_roidb,site_ix)
avgFP=[0.5,1,2,3,4,8,16,32,64]
iou_th=0.5

stack_box = all_boxes[site_ix]
result, valid_avgFP = sens_at_FP(stack_box, gt_boxes, avgFP, iou_th)
print('='*40)
for recall,fp in zip(result,valid_avgFP):
    print('Recall@%.1f=%.2f%%' % (fp, recall*100))
#TODO: when num of valid_avgFP < 6,is FROC correct?
print('Mean FROC is %.2f'% np.mean(np.array(result[:6])*100))
print('='*40)


sens, fp_per_img, scores_th = FROC(stack_box, gt_boxes, iou_th)

Loading cached gt_roidb from %s ./cache/PDS_AMGEN_20020408_22Cat_test_gt_roidb.pkl


/mnt/fast-data/mjc/envs/Lesions/lib/python3.6/site-packages/matplotlib/__init__.py:1405: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


======================================== 0
======================================== 500
======================================== 1000
======================================== 1500
======================================== 2000
======================================== 2500
======================================== 3000
======================================== 3500
======================================== 4000
======================================== 4500
======================================== 5000
======================================== 5500
======================================== 6000
======================================== 6500
======================================== 7000
======================================== 7500
======================================== 8000
======================================== 8500
======================================== 9000
FROC:FP in no-lesion-images:  587
Recall@0.5=34.67%
Recall@0.6=42.87%
Mean FROC is 38.77
FROC:FP in no-lesion-images:  587


In [3]:
site_list = [site_ix] #8 is liver
D_CT = {}
for i , aroidb in enumerate(roidb):
    dicom_path , png_name = os.path.split(aroidb['image'])
    slice_no , _= os.path.splitext(png_name)
    slice_no = int(slice_no)
    if slice_no != aroidb['slice_no']:
        print(dicom_path,slice_no,aroidb['slice_no'])

    segmentations = {}
    bboxes = {}
    for j in site_list:
        segmentations[j] = all_segms[j][i]
        bboxes[j] = all_boxes[j][i]

    if dicom_path not in D_CT:
        D_CT[dicom_path] = {}
        D_CT[dicom_path][slice_no] = [aroidb , bboxes , segmentations]
    else:
        D_CT[dicom_path][slice_no] = [aroidb , bboxes , segmentations]


# In[37]:

import pandas as pd

# DL_info = '../Inputs/PDS_AUTO_RECIST CIA-LAB Image and Contour 2020-10-01.xlsx'
# json_name = 'CUIMC20201027.json'

DL_info = '/mnt/fast-data/mjc/AutoRECIST/Inputs/PDS_AMGEN_20020408_CIA-LAB_Image_And_Contour_2020-10-21.xlsx'
# json_name = 'AMGEN_20020408_20201027.json'

# DL_info = '../Inputs/PDS_AMGEN_PRIME_CIA-LAB_Image_And_Contour_2020-10-21.xlsx'
# json_name = 'AMGEN_PRIME_20201027.json'

df = pd.read_excel(DL_info)

def pd_str_replace(df , col, ori, new):
    if isinstance(col , str):
        df[col] = df[col].str.replace(ori,new, case = False) 
    elif isinstance(col, list):
        for one in col:
            pd_str_replace(df , one, ori, new)
    else:
        raise('col instence should be str or list')

pd_str_replace(df, ['Image File Path' , 'Contour File Path'], "X:" , "/mnt/W-drive")
pd_str_replace(df, ['Image File Path' , 'Contour File Path'], r"\\" , "/")
pd_str_replace(df, ['Image File Path'], "/mnt/W-drive/ClinicalTrialDone/FNIH_VOLPACK", "/mnt/fast-disk1/mjc/AutoRecist/Inputs")
pd_str_replace(df, ['Image File Path'], "/mnt/W-drive/ClinicalTrials", "/mnt/fast-disk1/mjc/AutoRecist/Inputs")
pd_str_replace(df, ['Contour File Path'], "/mnt/W-drive/ConvWeasisToRaw", "/mnt/fast-disk1/mjc/AutoRecist/Inputs/ConvWeasisToRaw")


# In[52]:

sys.path.append('/mnt/fast-data/mjc/DeepLesion/Codes/CollectData/')

from print_weasis_raw_data import WeasisRawDataPrinter
from read_weasis_raw_data import WeasisRawFileReader


import os
import numpy as np
import pydicom
import matplotlib.pylab as plt


data_path = ""
metadata_path = "/mnt/fast-data/mjc/AutoRECIST/Outputs/Dicom_header"
if not os.path.exists(metadata_path):
    os.mkdir(metadata_path)


D_dir2header_df = {}


def get_dicom_header_df(image_dir , labels = []):
    global D_dir2header_df
    if image_dir in D_dir2header_df:
        return D_dir2header_df[image_dir]

    # image_dir = row['Image File Path']


    labels = ['ImageName','InstanceNumber',
            'BitsAllocated', 'BitsStored', 'Columns', 'HighBit', 
            'ImageOrientationPatient_0', 'ImageOrientationPatient_1', 'ImageOrientationPatient_2',
            'ImageOrientationPatient_3', 'ImageOrientationPatient_4', 'ImageOrientationPatient_5',
            'ImagePositionPatient_0', 'ImagePositionPatient_1', 'ImagePositionPatient_2',
            'Modality', 'PatientID', 'PhotometricInterpretation', 'PixelRepresentation',
            'PixelSpacing_0', 'PixelSpacing_1', 'RescaleIntercept', 'RescaleSlope', 'Rows', 'SOPInstanceUID',
            'SamplesPerPixel', 'SeriesInstanceUID', 'StudyID', 'StudyInstanceUID', 
            'WindowCenter', 'WindowWidth', 
        ] if not labels else labels

    data = {l: [] for l in labels}
    
    ctList = os.listdir(image_dir)
    ctList.sort()

    for image in ctList:
        if '.dcm' not in image:
            continue
        if os.path.getsize(os.path.join(image_dir, image)) < 5*1024:
            print('%s size < 5kb skiped!'%os.path.join(image_dir, image) )
            continue
        data["ImageName"].append(image)

        ds = pydicom.dcmread(os.path.join(image_dir, image))
        for metadata in ds.dir():
            if metadata not in data and metadata not in ['ImageOrientationPatient','ImagePositionPatient','PixelSpacing']:
                continue
            if metadata != "PixelData":
                metadata_values = getattr(ds, metadata)
                if type(metadata_values) == pydicom.multival.MultiValue and metadata not in ["WindowCenter", "WindowWidth"]:
                    for i, v in enumerate(metadata_values):
                        data[f"{metadata}_{i}"].append(v)  
                else:
                    if type(metadata_values) == pydicom.multival.MultiValue and metadata in ["WindowCenter", "WindowWidth"]:
                        data[metadata].append(metadata_values[0])
                    else:
                        data[metadata].append(metadata_values)

    df_image = pd.DataFrame(data).set_index("InstanceNumber")
    D_dir2header_df[image_dir] = df_image
    return df_image

In [4]:
df

,Pname,Image File Path,Contour File Path,Location,Uni,Perp,Bi,Volume
0,BAIJC,/mnt/fast-disk1/mjc/AutoRecist/Inputs/AMGEN/20...,/mnt/fast-disk1/mjc/AutoRecist/Inputs/ConvWeas...,liver,45.5731,36.56930,1666.576500,24536.30
1,BAIJC,/mnt/fast-disk1/mjc/AutoRecist/Inputs/AMGEN/20...,/mnt/fast-disk1/mjc/AutoRecist/Inputs/ConvWeas...,lung,12.8308,10.06720,129.170230,1344.73
2,BAIJC,/mnt/fast-disk1/mjc/AutoRecist/Inputs/AMGEN/20...,/mnt/fast-disk1/mjc/AutoRecist/Inputs/ConvWeas...,lung,15.1458,12.27960,185.984360,1586.98
3,BAIJC,/mnt/fast-disk1/mjc/AutoRecist/Inputs/AMGEN/20...,/mnt/fast-disk1/mjc/AutoRecist/Inputs/ConvWeas...,lung,10.7557,3.58525,38.561874,538.88
4,BAIJC,/mnt/fast-disk1/mjc/AutoRecist/Inputs/AMGEN/20...,/mnt/fast-disk1/mjc/AutoRecist/Inputs/ConvWeas...,lung,42.4736,31.22380,1326.187100,19320.60
...,...,...,...,...,...,...,...,...
3126,BAIXA,/mnt/fast-disk1/mjc/AutoRecist/Inputs/AMGEN/20...,/mnt/fast-disk1/mjc/AutoRecist/Inputs/ConvWeas...,liver,23.2746,13.19280,307.057130,7250.49
3127,BAIXA,/mnt/fast-disk1/mjc/AutoRecist/Inputs/AMGEN/20...,/mnt/fast-disk1/mjc/AutoRecist/Inputs/ConvWeas...,liver,16.8104,9.88845,166.228790,1557.17
3128,BAIXA,/mnt/fast-disk1/mjc/AutoRecist/Inputs/AMGEN/20...,/mnt/fast-disk1/mjc/AutoRecist/Inputs/ConvWeas...,liver,27.1977,17.68900,481.100100,5395.09
3129,BAIXA,/mnt/fast-disk1/mjc/AutoRecist/Inputs/AMGEN/20...,/mnt/fast-disk1/mjc/AutoRecist/Inputs/ConvWeas...,liver,17.4805,16.14270,282.182460,2574.10


In [5]:
!cp -v /mnt/fast-data/mjc/AutoRECIST/Codes/ScaleNAS/ScaleNASv1/tools/utils_metrics_3d.py .

‘/mnt/fast-data/mjc/AutoRECIST/Codes/ScaleNAS/ScaleNASv1/tools/utils_metrics_3d.py’ -> ‘./utils_metrics_3d.py’


In [ ]:
# In[53]:

from utils_metrics_3d import *
Metrics_vol = []
Missed_vol = []
previous_CT_ss = ''
for _,row in df.iterrows():

    if _%100==0 or _<10:
        print(_)
    if row.Location != ix2labelname(site_ix):
#         only calculate liver
        continue
    df_image = get_dicom_header_df( row['Image File Path'] )
    instanceNumber_list = df_image.index.to_list()
    D_z_index = instanceNumber2Matrix_z_index(instanceNumber_list)

    reader = WeasisRawFileReader()
    weasis_raw_data = reader.read_weasis_raw_file(row['Contour File Path'])

    ss = row['Image File Path']
    ss = ss.replace('/Inputs/' , '/Pngs/')
    if ss not in D_CT:
        print(ss , 'not in D_CT')

    mask_vol = initialize_mask_vol(weasis_raw_data , D_z_index)

    slice_list = weasis_raw_data.get_instance_number_array()
    for j, one in enumerate(slice_list):
        file_name = InstanceNumber2file_name(df_image, one)
        file_name = os.path.join( row['Image File Path'] , file_name)   
        mask = weasis_raw_data.get_mask_image_2d(j)
        mask_vol[D_z_index[one]] = mask

    if ss != previous_CT_ss:
        oneCT = D_CT[ss]
        vol_gt , vol_pred = get_gt_and_pred_vols( oneCT, site_list, mask_vol.shape, D_z_index, union_mask=False )
        previous_CT_ss = ss
    hit = vols_seg_results(mask_vol , vol_pred, CTname=ss, gt_keep_largest=1)
    Metrics_vol.extend(hit)


0
1
2
3
4
5
6
7
8
9
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400


In [11]:
# In[54]:

print(name)
df_metrics = pd.DataFrame(Metrics_vol, 
                          columns = ['file_name','igt','merge','#gt','#pred',
                                     'iou_score', 'dice_score', 'over_seg' , 'under_seg',
                                     'area_gt','area_pred','intersection','union']) 
df_metrics.describe([.05, .25, .5, .75, .95])

PDS_AMGEN_20020408_22Cat_test


,igt,merge,#gt,#pred,iou_score,dice_score,over_seg,under_seg,area_gt,area_pred,intersection,union
count,1376.000000,1376.0,1376.0,1376.000000,1376.000000,1376.000000,1376.000000,1376.000000,1.376000e+03,1.376000e+03,1.376000e+03,1.376000e+03
mean,1.003634,0.0,1.0,9.744186,0.268439,0.368423,62.298924,0.161172,2.485889e+04,2.200778e+05,2.255948e+04,2.223773e+05
std,0.060193,0.0,0.0,6.059580,0.240168,0.292585,519.826265,0.265672,1.140864e+05,6.735954e+05,1.059485e+05,6.758965e+05
min,1.000000,0.0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.700000e+01,0.000000e+00,0.000000e+00,1.800000e+01
5%,1.000000,0.0,1.0,2.000000,0.000929,0.001856,0.000000,0.000000,1.460000e+02,6.075000e+01,2.950000e+01,3.107500e+02
25%,1.000000,0.0,1.0,5.000000,0.034363,0.066442,0.580710,0.006613,6.945000e+02,1.777250e+03,4.987500e+02,2.293750e+03
50%,1.000000,0.0,1.0,8.000000,0.218496,0.358633,2.016682,0.039620,2.662000e+03,1.717300e+04,2.205500e+03,1.951050e+04
75%,1.000000,0.0,1.0,14.000000,0.476947,0.645856,17.014892,0.179098,1.194850e+04,1.545850e+05,1.026475e+04,1.547928e+05
95%,1.000000,0.0,1.0,20.000000,0.680027,0.809542,182.883353,0.954900,9.314275e+04,9.360840e+05,8.976650e+04,9.361225e+05
max,2.000000,0.0,1.0,24.000000,0.827836,0.905810,15032.833333,1.000000,3.159524e+06,5.361926e+06,3.003384e+06,5.518066e+06


In [12]:
# In[55]:

print(name)
df_miss = pd.DataFrame(Missed_vol, 
                          columns = ['file_name','igt','merge','#gt','#pred',
                                     'iou_score', 'dice_score', 'over_seg' , 'under_seg',
                                     'area_gt','area_pred','intersection','union']) 
df_miss.describe([.05, .25, .5, .75, .95])

PDS_AMGEN_20020408_22Cat_test


,file_name,igt,merge,#gt,#pred,iou_score,dice_score,over_seg,under_seg,area_gt,area_pred,intersection,union
count,0,0,0,0,0,0,0,0,0,0,0,0,0
unique,0,0,0,0,0,0,0,0,0,0,0,0,0
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
df_metrics.to_csv('%s_liver_seg_metrics_vol_20210719.csv'%name)

In [13]:
name

'PDS_AMGEN_20020408_22Cat_test'

In [ ]:
stop

In [ ]:
# In[ ]:

SHOW_LABEL = True
SHOW_BOX = False
SHOW_MASK = True
SHOW_UNION_MASK= False
SHOW_MASK_LABEL = True
SHOW_GT_MASK = True

# site_list = range(1,len(all_boxes))
site_list = [site_ix] #8 is liver
savepath = '/mnt/fast-disk1/mjc/AutoRecist/Outputs/ScaleNAS/%s/Images_%s_UnionMask_liver'%(mask_name,name)
if not os.path.exists(savepath):
    os.makedirs(savepath)

for i , aroidb in enumerate(roidb):

    image_path = os.path.join(aroidb['image'])
    [HU1, HU2 ] = aroidb['windows']

    image = load_image(image_path, HU1, HU2)
    height,width = image.shape
#     plt.imshow(image)
    image = np.dstack((image,image,image))
    if SHOW_GT_MASK:
        ix = [a for a,b in enumerate(aroidb['gt_classes']) if int(b) in site_list]
        contours = [ aroidb['segms'][int(kk)] for kk in ix ]
        if not contours:
            continue
        colors = [0,0,255]
        for c in contours:
            c = [ np.int64( np.reshape(c,(-1,2)) ) for c in c]
            if len(c):
                image = cv2.drawContours(image, c, -1, colors, 1)
#             c = np.reshape(c,(-1,2))
#             if c.shape[0]:
#                 image = cv2.drawContours(image, [np.int64( c  )], -1, colors, 1)
    
    if SHOW_MASK:
        for j in site_list:
            contours = all_segms[j][i]
            colors = compute_colors_for_labels(j)
            for c in contours:
                c = np.reshape(c,(-1,2))
                if c.shape[0]:
                    image = cv2.drawContours(image, [np.int64( c  )], -1, colors, 1)
    

    if SHOW_UNION_MASK:
        for j in site_list:
            contours = all_segms[j][i]
            cc = [ contour.flatten().tolist() for contour in contours if len(contour)!=0]
            contours = union_ploys(cc , height, width)
            colors = compute_colors_for_labels(j)
            label = ix2labelname(j)

            for c in contours:
                c = np.reshape(c,(-1,2))
                if c.shape[0]:
                    image = cv2.drawContours(image, [np.int64( c  )], -1, colors, 1)
                    if SHOW_MASK_LABEL:
                        x,y,_,_ =ploy2boxes(c)
                        template = "{}"
                        if len(label)>=9:
                            label=label[:3]+label[-3:]
            
                        s = template.format(label)
                        cv2.putText(
                            image, s, (x, y), cv2.FONT_HERSHEY_SIMPLEX, .5, colors, 1
                        )        

    if SHOW_BOX:
        for j in site_list:
            colors = compute_colors_for_labels(j)
            for box in all_boxes[j][i]:
                
                if box.shape[0]:
                    top_left, bottom_right = box[:2].tolist(), box[2:4].tolist()
                    image = cv2.rectangle(image, tuple(np.int64(top_left).tolist()), tuple(np.int64(bottom_right).tolist()), colors, 1)
                
            
    
    cv2.imwrite( os.path.join(savepath, '%06d.png'%aroidb['id'] ), image )

In [ ]:

all_segms[8][5]


# In[ ]:

all_boxes[8][5]


# In[ ]:




# In[ ]:


In [ ]:
whos